# Audio Classification

The standard audio classification architecture is motivated by the nature of the task; we want to transform a sequence of audio inputs (i.e. our input audio array) into a single class label prediction. Encoder-only models first map the input audio sequence into a sequence of hidden-state representations by passing the inputs through a transformer block. The sequence of hidden-state representations is then mapped to a class label output by taking the mean over the hidden-states, and passing the resulting vector through a linear classification layer. Hence, there is a preference for encoder-only models for audio classification.

Decoder-only models introduce unnecessary complexity to the task, since they assume that the outputs can also be a sequence of predictions (rather than a single class label prediction), and so generate multiple outputs. Therefore, they have slower inference speed and tend not to be used. Encoder-decoder models are largely omitted for the same reason. These architecture choices are analogous to those in NLP, where encoder-only models such as BERT are favoured for sequence classification tasks, and decoder-only models such as GPT reserved for sequence generation tasks.

## Keyword spotting (KWS) 
KWS is the task of identifying a keyword in a spoken utterance. The set of possible keywords forms the set of predicted class labels. Hence, to use a pre-trained keyword spotting model, you should ensure that your keywords match those that the model was pre-trained on. Below, we’ll introduce two datasets and models for keyword spotting.

MINDS-14: https://huggingface.co/datasets/PolyAI/minds14

contains recordings of people asking an e-banking system questions in several languages and dialects, and has the intent_class for each recording. We can classify the recordings by intent of the call.

In [1]:
from datasets import load_dataset

minds = load_dataset("PolyAI/minds14", name="en-AU", split="train")

XLS-R model fine-tuned on MINDS-14 for approximately 50 epochs. It achieves 90% accuracy over all languages from MINDS-14 on the evaluation set: https://huggingface.co/anton-l/xtreme_s_xlsr_300m_minds14

In [2]:
from transformers import pipeline

classifier = pipeline(
    "audio-classification",
    model="anton-l/xtreme_s_xlsr_300m_minds14",
)

In [3]:
minds[0]["path"]

'/home/lkk/.cache/huggingface/datasets/downloads/extracted/982a9a59db1777f93fb67e8ae85fd2066e0e4f157408e90dd6b406a675e4ccf1/en-AU~PAY_BILL/response_4.wav'

In [4]:
result=classifier(minds[0]["path"])
result

[{'score': 0.9623648524284363, 'label': 'pay_bill'},
 {'score': 0.028678400442004204, 'label': 'freeze'},
 {'score': 0.003429617965593934, 'label': 'card_issues'},
 {'score': 0.0020604885648936033, 'label': 'abroad'},
 {'score': 0.0008625672780908644, 'label': 'high_value_payment'}]

## Speech Commands
Speech Commands is a dataset of spoken words designed to evaluate audio classification models on simple command words. The dataset consists of 15 classes of keywords, a class for silence, and an unknown class to include the false positive. The 15 keywords are single words that would typically be used in on-device settings to control basic tasks or launch other processes.

https://huggingface.co/datasets/speech_commands

In [5]:
speech_commands = load_dataset(
    "speech_commands", "v0.02", split="validation", streaming=True
)
sample = next(iter(speech_commands))

In [6]:
sample

{'file': 'backward/0d82fd99_nohash_2.wav',
 'audio': {'path': 'backward/0d82fd99_nohash_2.wav',
  'array': array([-9.15527344e-05,  6.10351562e-05,  6.10351562e-05, ...,
          2.68859863e-02,  1.46179199e-02,  1.67846680e-03]),
  'sampling_rate': 16000},
 'label': 30,
 'is_unknown': True,
 'speaker_id': '0d82fd99',
 'utterance_id': 2}

We’ll load an official Audio Spectrogram Transformer checkpoint fine-tuned on the Speech Commands dataset, under the namespace "MIT/ast-finetuned-speech-commands-v2":

In [7]:
classifier = pipeline(
    "audio-classification", model="MIT/ast-finetuned-speech-commands-v2"
)
classifier(sample["audio"].copy())

[{'score': 0.9999892711639404, 'label': 'backward'},
 {'score': 1.7504888774055871e-06, 'label': 'happy'},
 {'score': 6.703020858367381e-07, 'label': 'follow'},
 {'score': 5.805890168630867e-07, 'label': 'stop'},
 {'score': 5.614535893982975e-07, 'label': 'up'}]

In [8]:
from IPython.display import Audio

Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

## FLEURS
FLEURS (Few-shot Learning Evaluation of Universal Representations of Speech) is a dataset for evaluating speech recognition systems in 102 languages, including many that are classified as ‘low-resource’. Take a look at the FLEURS dataset card on the Hub and explore the different languages that are present: google/fleurs.

In [9]:
fleurs = load_dataset("google/fleurs", "all", split="validation", streaming=True)
sample = next(iter(fleurs))

we’ll use a version of Whisper fine-tuned on the FLEURS dataset, which is currently the most performant LID model on the Hub:

In [10]:
classifier = pipeline(
    "audio-classification", model="sanchit-gandhi/whisper-medium-fleurs-lang-id"
)

In [11]:
classifier(sample["audio"])

[{'score': 0.9999330043792725, 'label': 'Afrikaans'},
 {'score': 7.093003659974784e-06, 'label': 'Northern-Sotho'},
 {'score': 4.269141300028423e-06, 'label': 'Icelandic'},
 {'score': 3.266111207267386e-06, 'label': 'Danish'},
 {'score': 3.258066044509178e-06, 'label': 'Cantonese Chinese'}]

## Zero-Shot Audio Classification

Zero-shot audio classification is a method for taking a pre-trained audio classification model trained on a set of labelled examples and enabling it to be able to classify new examples from previously unseen classes.

Transformers supports one kind of model for zero-shot audio classification: the CLAP model. CLAP is a transformer-based model that takes both audio and text as inputs, and computes the similarity between the two. If we pass a text input that strongly correlates with an audio input, we’ll get a high similarity score. Conversely, passing a text input that is completely unrelated to the audio input will return a low similarity.

We can use this similarity prediction for zero-shot audio classification by passing one audio input to the model and multiple candidate labels. The model will return a similarity score for each of the candidate labels, and we can pick the one that has the highest score as our prediction.

take an example where we use one audio input from the Environmental Speech Challenge (ESC) dataset:  https://huggingface.co/datasets/ashraq/esc50

In [12]:
dataset = load_dataset("ashraq/esc50", split="train", streaming=True)
audio_sample = next(iter(dataset))["audio"]["array"]

Repo card metadata block was not found. Setting CardData to empty.


In [13]:
candidate_labels = ["Sound of a dog", "Sound of vacuum cleaner"]

In [14]:
classifier = pipeline(
    task="zero-shot-audio-classification", model="laion/clap-htsat-unfused"
)
classifier(audio_sample, candidate_labels=candidate_labels)

[{'score': 0.9997242093086243, 'label': 'Sound of a dog'},
 {'score': 0.00027583108749240637, 'label': 'Sound of vacuum cleaner'}]

In [15]:
#listen to the audio sample
Audio(audio_sample, rate=16000)

## Music Classification
a step-by-step guide on fine-tuning an encoder-only transformer model for music classification

ref: https://huggingface.co/learn/audio-course/chapter4/fine-tuning

To train our model, we’ll use the GTZAN dataset: https://huggingface.co/datasets/marsyas/gtzan, which is a popular dataset of 1,000 songs for music genre classification. Each song is a 30-second clip from one of 10 genres of music, spanning disco to metal. 

In [1]:
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

Found cached dataset gtzan (C:/Users/lkk68/.cache/huggingface/datasets/marsyas___gtzan/all/0.0.0/701395d625fb4762c27eef42c9488d843cc15869b399f7d207b07306a8b09762)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

GTZAN doesn’t provide a predefined validation set, so we’ll have to create one ourselves. The dataset is balanced across genres, so we can use the train_test_split() method to quickly create a 90/10 split as follows:

In [2]:
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [3]:
gtzan["train"][0]

{'file': 'C:\\Users\\lkk68\\.cache\\huggingface\\datasets\\downloads\\extracted\\3b871588f35b2bcee38b7cb76eab303286c1c62de2566e4eaf0bc11405f35fd0\\genres\\pop\\pop.00098.wav',
 'audio': {'path': 'C:\\Users\\lkk68\\.cache\\huggingface\\datasets\\downloads\\extracted\\3b871588f35b2bcee38b7cb76eab303286c1c62de2566e4eaf0bc11405f35fd0\\genres\\pop\\pop.00098.wav',
  'array': array([ 0.10720825,  0.16122437,  0.28585815, ..., -0.22924805,
         -0.20629883, -0.11334229]),
  'sampling_rate': 22050},
 'genre': 7}

We can also see the genre is represented as an integer, or class label, which is the format the model will make it’s predictions in. Let’s use the int2str() method of the genre feature to map these integers to human-readable names:

In [4]:
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn(gtzan["train"][0]["genre"])

'pop'

In [20]:
import gradio as gr


def generate_audio():
    example = gtzan["train"].shuffle()[0]
    audio = example["audio"]
    return (
        audio["sampling_rate"],
        audio["array"],
    ), id2label_fn(example["genre"])


with gr.Blocks() as demo:
    with gr.Column():
        for _ in range(4):
            audio, label = generate_audio()
            output = gr.Audio(audio, label=label)

demo.launch(debug=True)

/home/lkk/miniconda3/envs/mypy310/lib/python3.10/site-packages/gradio/processing_utils.py:354: UserWarning: Trying to convert audio automatically from float64 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In this domain, pretraining is typically carried out on large amounts of unlabeled audio data, using datasets like LibriSpeech and Voxpopuli.

https://huggingface.co/datasets/librispeech_asr: LibriSpeech is a corpus of approximately 1000 hours of 16kHz read English speech, automatic-speech-recognition, audio-speaker-identification: The dataset can be used to train a model for Automatic Speech Recognition (ASR). The model is presented with an audio file and asked to transcribe the audio file to written text. The most common evaluation metric is the word error rate (WER).

VoxPopuli (https://huggingface.co/datasets/facebook/voxpopuli) is a large-scale multilingual speech corpus for representation learning, semi-supervised learning and interpretation. The raw data is collected from 2009-2020 European Parliament event recordings.

Although models like Wav2Vec2 and HuBERT are very popular, we’ll use a model called DistilHuBERT. This is a much smaller (or distilled) version of the HuBERT model

The conversion from audio to the input format is handled by the feature extractor of the model. Transformers provides a convenient AutoFeatureExtractor class that can automatically select the correct feature extractor for a given model. To see how we can process our audio files, let’s begin by instantiating the feature extractor for DistilHuBERT from the pre-trained checkpoint:

In [5]:
from transformers import AutoFeatureExtractor

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

c:\Users\lkk68\.conda\envs\mycondapy39\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lkk68\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Since the sampling rate of the model and the dataset are different, we’ll have to resample the audio file to 16,000 Hz before passing it to the feature extractor. We can do this by first obtaining the model’s sample rate from the feature extractor:

In [6]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

resample the dataset using the cast_column() method 

In [7]:
from datasets import Audio

gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

Datasets will resample the audio file on-the-fly when we load each audio sample:

In [8]:
gtzan["train"][0]

{'file': 'C:\\Users\\lkk68\\.cache\\huggingface\\datasets\\downloads\\extracted\\3b871588f35b2bcee38b7cb76eab303286c1c62de2566e4eaf0bc11405f35fd0\\genres\\pop\\pop.00098.wav',
 'audio': {'path': 'C:\\Users\\lkk68\\.cache\\huggingface\\datasets\\downloads\\extracted\\3b871588f35b2bcee38b7cb76eab303286c1c62de2566e4eaf0bc11405f35fd0\\genres\\pop\\pop.00098.wav',
  'array': array([ 0.08735093,  0.20183387,  0.47908676, ..., -0.1874318 ,
         -0.23294398, -0.13517427]),
  'sampling_rate': 16000},
 'genre': 7}

A defining feature of Wav2Vec2 and HuBERT like models is that they accept a float array corresponding to the raw waveform of the speech signal as an input. This is in contrast to other models, like Whisper, where we pre-process the raw audio waveform to spectrogram format.

the audio data is in the right format, but we’ve imposed no restrictions on the values it can take. For our model to work optimally, we want to keep all the inputs within the same dynamic range. This is going to make sure we get a similar range of activations and gradients for our samples, helping with stability and convergence during training. To do this, we normalise our audio data, by rescaling each sample to zero mean and unit variance, a process called feature scaling. It’s exactly this feature normalisation that our feature extractor performs!

We can take a look at the feature extractor in operation by applying it to our first audio sample. First, let’s compute the mean and variance of our raw audio data:

In [9]:
import numpy as np

sample = gtzan["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: 0.000185, Variance: 0.0493


We can see that the mean is close to zero already, but the variance is closer to 0.05. If the variance for the sample was larger, it could cause our model problems, since the dynamic range of the audio data would be very small and thus difficult to separate. Let’s apply the feature extractor and see what the outputs look like:

In [10]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

inputs keys: ['input_values', 'attention_mask']
Mean: -7.03e-09, Variance: 1.0


Our feature extractor returns a dictionary of two arrays: input_values and attention_mask. The input_values are the preprocessed audio inputs that we’d pass to the HuBERT model. The attention_mask is used when we process a batch of audio inputs at once - it is used to tell the model where we have padded inputs of different lengths.

We can see that the mean value is now very much closer to zero, and the variance bang-on one! This is exactly the form we want our audio samples in prior to feeding them to the HuBERT model.

The last thing to do is define a function that we can apply to all the examples in the dataset. Since we expect the audio clips to be 30 seconds in length, we’ll also truncate any longer clips by using the max_length and truncation arguments of the feature extractor as follows:

In [11]:
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

The .map() method supports working with batches of examples, which we’ll enable by setting batched=True. The default batch size is 1000

In [12]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
gtzan_encoded

Map:   0%|          | 0/899 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

To simplify the training, we’ve removed the audio and file columns from the dataset. The input_values column contains the encoded audio files, the attention_mask a binary mask of 0/1 values that indicate where we have padded the audio input, and the genre column contains the corresponding labels (or targets). To enable the Trainer to process the class labels, we need to rename the genre column to label:

In [13]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

Finally, we need to obtain the label mappings from the dataset. This mapping will take us from integer ids (e.g. 7) to human-readable class labels (e.g. "pop") and back again. In doing so, we can convert our model’s integer id prediction into human-readable format, enabling us to use the model in any downstream application. We can do this by using the int2str() method as follows:

In [14]:
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["7"]

'pop'

In [15]:
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['projector.bias', 'classifier.bias', 'classifier.weight', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
from huggingface_hub import notebook_login

notebook_login()

In [16]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10

training_args = TrainingArguments(
    f"./data/{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
)

In [17]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [18]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/lkk688/distilhubert-finetuned-gtzan into local empty directory.


Download file pytorch_model.bin:   0%|          | 8.00k/90.4M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 4.00k/4.00k [00:00<?, ?B/s]

Clean file training_args.bin:  25%|##5       | 1.00k/4.00k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/90.4M [00:00<?, ?B/s]

c:\Users\lkk68\.conda\envs\mycondapy39\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1130 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 │   compute_metrics=compute_metrics,                                                        │
│   10 )                                                                                           │
│   11                                                                                             │
│ ❱ 12 trainer.train()                                                                             │
│   13                                                                                             │
│                                                                                                  │
│ c:\Users\lkk68\.conda\envs\mycondapy39\lib\site-packages\transformers\trainer.py:1645 in train   │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ c:\Users\lkk68\.conda\envs\mycondapy39\lib\site-packages\transformers\trainer.py:1938 in         │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ c:\Users\lkk68\.conda\envs\mycondapy39\lib\site-packages\transformers\trainer.py:2759 in         │
│ training_step                                                                                    │
│                                                                                                  │
│   2756 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2757 │   │                                                                                     │
│   2758 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2759 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2760 │   │                                                                                     │
│   2761 │   │   if self.args.n_gpu > 1:                                                           │
│   2762 │   │   │   loss = loss.mean()  # mean() to average 

: 

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "audio-classification", model="lkk688/distilhubert-finetuned-gtzan"
)